# Lecture 01: Convex Hulls

*Author: Jan Erik Swiadek*

This notebook serves as supplementary learning material for the lecture **Geometric Algorithms** by Prof. Dr. Kevin Buchin.
It showcases implementations of the algorithms and data structures presented in the lecture, elaborates on some practical considerations concerning their use. 
Furthermore, it includes interactive visualisations (and animations?) of the algorithms.

## Table of Contents

1. [Introduction](#1-introduction)  
2. [Imports](#2-imports)  
3. [Algorithms](#3-algorithms)  
    3.1. [Naive Hull](#31-naive-hull)  
    3.2. [Graham Scan](#32-graham-scan)  
    3.3. [Gift Wrapping](#33-gift-wrapping)  
    3.4. [Chan's Hull](#34-chans-hull)  
4. [Conclusion](#4-conclusion)

## 1. Introduction

A set of points is *convex* if for any two points in the set, their connecting line segment is part of the set.
The *convex hull* of a set of points is the smallest convex set subsuming the given set.
It's a convex polytope, which vertices are in the set of points.

## 2. Imports

Import from a central module for geometry functions.
This will get a module later as well.

In [ ]:
from geometry import Point, PointRef, Orientation, Polyline

from typing import Iterator, Optional

Import visualisations.

In [ ]:
from visualisation import Visualisation
vis = Visualisation()
points = [Point(10,10), Point(100,50), Point(250,25), Point(100,100), Point(200,200), Point(300,10), Point(400,10)]
vis.add_points(points)
vis.register_instance("Test instance", points)

## 3. Algorithms

As convex hull is a fundamental problem in computational geometry, there are many algorithms that have been developed for solving it.
The lecture presented four of them.

### 3.1 Naive Hull

The first algorithm from the lecture makes use of the fact that for any edge of the convex hull, all other points lie to one side of the edge.
In the lecture, they are supposed to lie on the left of the edge.
However, if we consider *degenerate cases*, which are common edge cases often left out of descriptions of geometric algorithms , this criterion is not sufficient.

There could be three collinear points, i.e. they all lie on the same line and none of the points is to the left or right of an edge between the other two points.
In that case, we want to include the outer two points in the convex hull, while the middle point is left out.
To achieve that, all other points are supposed to lie either on left side of the edge or directly on the edge between the two points.
If a point is found that violates both those criterions, the edge is marked as invalid and not included in the convex hull.
(see CG)

We assume no two same points in the input, that's why we take it as a set.

In [ ]:
def naive_hull(points: set[Point]) -> Polyline:
    points = list(points)
    if len(points) <= 2:
        return Polyline(points)
    
    edges: dict[Point, Point] = {}
    for p in points:
        for q in filter(lambda q: q != p, points):
            valid = True
            for r in filter(lambda r: r != p and r != q, points):
                orientation = r.orientation(p, q)
                if orientation is not Orientation.LEFT and orientation is not Orientation.BETWEEN:
                    valid = False
                    break
            if valid:
                edges[p] = q
    
    hull = Polyline()
    if edges:
        # Construct a list of ordered vertices from the included edges.
        first_point, next_point = edges.popitem()
        hull.append(first_point)
        while next_point is not None and next_point != first_point:
            hull.append(next_point)
            next_point = edges.pop(next_point, None)
    
    return hull

Because of the naive nested loop, the algorithm's worst-case complexity is in *O(n³)*
Let's register the algorithm for the visualisation.

In [ ]:
print(naive_hull(points))

In [ ]:
vis.register_algorithm("Naive hull", naive_hull)

Let's test it.
You need to run the cells so execute the notebook on Binder or locally.
You can add points by clicking.
The *Degeneracy test instance* includes degenerate cases, which are handled.
But the *robustness test instance* shows that it's still not correct in every case.
Try it out.
Why is that?

In [ ]:
vis.display()

It's because floating point arithmetic is not exact and rounding errors can lead to errors.
But using correct arithmetic is unfortunately very slow. (Checkbox for swap? Or another test set?)
(see CG)

**Takeaway**:
Inefficient.
Even a simple algorithm can have correctness / robustness issues.

### 3.2 Graham Scan

The second algorithm from the lecture is called *Graham Scan*.
It sorts the points by ascending x-coordinate and scans them in this order to create the upper boundary of the convex hull.
After that, it scans the points in reversed order to create the lower boundary.

The version from the lectures handles collinear points correctly since collinear points do not make a turn.
Not making a right turn is equivalent to the middle point not being on left of the line segment.
On the other hand, it doesn't consider how points with the same x-coordinate are sorted.
As it turns out, they need to be sorted by ascending y-coordinate.

In [ ]:
def graham_scan(points: set[Point]) -> Polyline:
    points = list(points)
    if len(points) <= 2:
        return Polyline(points)
    
    sorted_points = sorted(points, key = lambda p: (p.x, p.y))
    upper_hull = graham_half_scan(sorted_points)

    sorted_points.reverse()
    lower_hull = graham_half_scan(sorted_points)

    return upper_hull + lower_hull[1:-1]

def graham_half_scan(points: list[Point]) -> Polyline:
    hull: Polyline = Polyline(points[:2])
    for p in points[2:]:
        hull.append(p)
        while len(hull) > 2 and hull[-2].orientation(hull[-3], hull[-1]) is not Orientation.LEFT:
            del hull[-2]
    
    return hull

The runtime is *O(n* log *(n))*, which is worst-case optimal (see Mount).
We register this algorithm as well.

In [ ]:
print(graham_scan(points))

In [ ]:
vis.register_algorithm("Graham scan", graham_scan)

Have fun!

In [ ]:
vis.display()

It also passes the robustness test, meaning Graham scan is more robust than Naive Hull (see CG).
It doesn't mean it doesn't have robustness issues at all.

**Takeaway:**
Sometimes a more efficient and robust algorithm can still be very simple.
In fact, the implementation of Graham Scan is less code than Naive Hull.

### 3.3 Gift Wrapping

The next algorithm from the lecture is *Gift Wrapping*, also called *Jarvis's March*, and it's output-sensitive.
Meaning, its runtime depends on the number *h* of points in the convex hull.

Explain is_better, which also contains the degeneracy handling

It has a termination safeguard.

In [ ]:
def gift_wrapping(points: set[Point]) -> Polyline:
    points = list(points)
    if len(points) <= 2:
        return Polyline(points)
    
    first_point = max(points, key = lambda p: (p.x, p.y))
    wrapper = Wrapper(first_point)

    hull = Polyline([first_point])
    while True:
        next_point = wrapper.get_next_point(points)
        if next_point == first_point or len(hull) == len(points):
            break
        hull.append(next_point)
    
    return hull


class Wrapper:
    def __init__(self, current_point: Point):
        self.current_point = current_point
    
    def get_next_point(self, points: list[Point]) -> Point:
        points_iterator: Iterator[Point] = filter(lambda p: p != self.current_point, points)
        next_point = next(points_iterator)

        for p in points_iterator:
            if self.is_better(p, next_point):
                next_point = p
        
        self.current_point = next_point
        return next_point

    def is_better(self, p: Point, q: Point) -> bool:
        orientation = p.orientation(self.current_point, q)
        return orientation is Orientation.LEFT or orientation is Orientation.BEHIND_TARGET

No idea about robustness yet.
Test it after registering.

In [ ]:
vis.register_algorithm("Gift wrapping", gift_wrapping)

Have fun!

In [ ]:
vis.display()

**Takeaway:**
Algorithmic complexity can depend on the size of the output instead of just the input.
Such algorithms might be beneficial in specific scenarios, though they are hard to judge for general cases.

### 3.4 Chan's Hull

Best of both worlds from a theoretical perspective.
But how does it perform in practice?

In [ ]:
def chans_hull(points: set[Point]) -> Polyline:
    points = list(points)
    if len(points) <= 2:
        return Polyline(points)

    hull = None
    t = 0
    while hull is None:
        m = min(len(points), 2**(2**t))
        hull = chans_hull_m(points, m)
        t += 1

    return hull

def chans_hull_m(points: list[Point], m: int) -> Optional[Polyline]:
    mini_hulls = []
    mini_hull_points = []
    for i in range(0, len(points), m):
        j = min(len(points), i+m)
        mini_hull = graham_scan(points[i:j])
        mini_hulls.append(mini_hull)
        for position in range(0, len(mini_hull)):
            mini_hull_points.append(PointRef(mini_hull, position))

    first_point = max(mini_hull_points, key = lambda p: (p.x, p.y))
    wrapper = ChanWrapper(first_point)
    hull = Polyline([first_point])
    for _ in range(0, m):
        candidates = []
        for mini_hull in mini_hulls:
            candidates.extend(wrapper.get_candidates(mini_hull))
        next_point = wrapper.get_next_point(candidates)
        if next_point == first_point:
            return hull
        hull.append(next_point)
        
    return None

class ChanWrapper(Wrapper):
    def __init__(self, current_point: PointRef):
        super(ChanWrapper, self).__init__(current_point)

    # Can return more than one candidate but at most five which is asymptotically constant.
    def get_candidates(self, mini_hull: list[Point]) -> list[PointRef]:
        if self.current_point.is_in_container(mini_hull):
            if len(mini_hull) == 1:
                return []
            next_position = (self.current_point.get_position() + 1) % len(mini_hull)
            return [PointRef(mini_hull, next_position)]

        l, mb, m, ma, r = self.calculate_indexes(mini_hull, 0, len(mini_hull) - 1)
        while l <= mb < m < ma <= r:
            m_better_than_before = self.is_better(mini_hull[m], mini_hull[mb])
            m_better_than_after = self.is_better(mini_hull[m], mini_hull[ma])
            if m_better_than_before and m_better_than_after:
                return [PointRef(mini_hull, m)]

            m_better_than_l = self.is_better(mini_hull[m], mini_hull[l])
            r_better_than_l = self.is_better(mini_hull[r], mini_hull[l])
            if r_better_than_l:
                if m_better_than_l and not m_better_than_before:
                    r = mb
                else:
                    l = m
            else:
                if m_better_than_l and m_better_than_before:
                    l = ma
                else:
                    r = m
            l, mb, m, ma, r = self.calculate_indexes(mini_hull, l, r)

        return [PointRef(mini_hull, pos) for pos in range(l, r+1)]
    
    # Handles all the degenerate edge cases. Don't read it, lest it will haunt you in your nightmares.
    def calculate_indexes(self, mini_hull: list[Point], l: int, r: int) -> tuple[int, int, int, int, int]:
        if l != r and self.has_same_angle(mini_hull[l], mini_hull[r]):
            if self.is_better(mini_hull[r], mini_hull[l]):
                l += 1
            else:
                r -= 1
        if r - l < 2:
            return l, 0, 0, 0, r

        m = int(l + (r - l) / 2)
        mb = m - 1
        ma = m + 1
        if self.has_same_angle(mini_hull[m], mini_hull[mb]):
            if self.is_better(mini_hull[mb], mini_hull[m]):
                m -= 1
            mb -= 1
        elif self.has_same_angle(mini_hull[m], mini_hull[ma]):
            if self.is_better(mini_hull[ma], mini_hull[m]):
                m += 1
            ma += 1
            
        return l, mb, m, ma, r

    def has_same_angle(self, p: Point, q: Point) -> bool:
        orient = p.orientation(self.current_point, q)
        return orient is Orientation.BETWEEN or orient is Orientation.BEHIND_TARGET

In [ ]:
print(chans_hull([Point(0,0), Point(1,0.5), Point(2.5,0.25), Point(1,1), Point(2,2), Point(3,0), Point(4,0), Point(1,2), Point(3,1.5)]))

In [ ]:
print(chans_hull([Point(0,3), Point(10,10), Point(20,5)]))

In [ ]:
vis.register_algorithm("Chan's hull", chans_hull)

In [ ]:
vis.display()

## 4. Conclusion